In [1]:
import trafilatura
import requests
import lxml
from elasticsearch import Elasticsearch, helpers


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
kenh = {
    "life": ["https://kenh14.vn/xem-an-choi.chn",
             "https://kenh14.vn/doi-song/nhan-vat.chn",
             "https://kenh14.vn/doi-song/tram-yeu.chn"],
    "news": ["https://kenh14.vn/xa-hoi/song-xanh.chn",
             "https://kenh14.vn/xa-hoi/phap-luat.chn",
             "https://kenh14.vn/xa-hoi/nong-tren-mang.chn"],
    "world": ["https://kenh14.vn/the-gioi/di.chn",
              "https://kenh14.vn/the-gioi/chum-anh.chn",
              "https://kenh14.vn/the-gioi/kham-pha.chn"]
}


In [3]:
kenhLinks = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in kenh.items():
    for url in value:
        response = requests.get(url)
        tree = lxml.html.fromstring(response.content)
        title_elem = tree.cssselect('.knswli-title a')
        for elem in title_elem:
            kenhLinks[key].append("https://kenh14.vn"+elem.get('href'))

print(kenhLinks)

{'life': ['https://kenh14.vn/mung-xiaomi-fan-festival-fpt-shop-giam-den-45-cho-san-pham-thuoc-he-sinh-thai-xiaomi-20220406165602534.chn', 'https://kenh14.vn/samfans-co-the-nang-cap-len-samsung-s22-series-ngay-bay-gio-20220406123220564.chn', 'https://kenh14.vn/tu-co-sinh-vien-ha-noi-mot-minh-vao-sai-gon-lap-nghiep-den-chu-chuoi-cua-hang-cho-me-va-be-20220405220223483.chn', 'https://kenh14.vn/den-sai-gon-dung-quen-loat-mon-ngon-dinh-dam-quen-loi-ve-20220405220456551.chn', 'https://kenh14.vn/meo-su-dung-may-hut-am-khong-khi-cho-nha-luon-thong-thoang-20220405174915295.chn', 'https://kenh14.vn/hint-khoe-dang-dep-da-mua-he-cua-nha-tocotoco-co-lam-ban-hao-huc-20220405142229624.chn', 'https://kenh14.vn/et-o-et-chinh-thuc-phat-dong-bidv-check-in-challenge-vi-vu-khap-noi-am-giai-thuong-den-400-trieu-dong-20220405120255625.chn', 'https://kenh14.vn/snack-khoai-tay-lays-chinh-phuc-trai-tim-gioi-tre-viet-voi-nguon-khoai-tay-noi-dia-20220404170545249.chn', 'https://kenh14.vn/giam-doc-dieu-hanh-do-ven

In [4]:
kenhContent = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in kenhLinks.items():
    for url in value:
        if(url.startswith('https://kenh14.vn/')):
            id = url.split('-')[-1].replace('.chn', '')
            downloaded = trafilatura.fetch_url(url)
            tree = lxml.html.fromstring(str(downloaded))
            date_elem = len(tree.cssselect(
                ".kbwcm-time")) and tree.cssselect(".kbwcm-time")[0].text or ""
            content = trafilatura.extract(downloaded)
            content = content.replace('\n', ' ')
            item = {"domain": "https://kenh14.vn/", "URL": url,
                    "content": content, "date": date_elem, "id": id}
            kenhContent[key].append(item)

print(len(kenhContent['life'])+len(kenhContent['news'])+len(kenhContent['world']))
print(kenhContent['life'][0])

50
{'domain': 'https://kenh14.vn/', 'URL': 'https://kenh14.vn/mung-xiaomi-fan-festival-fpt-shop-giam-den-45-cho-san-pham-thuoc-he-sinh-thai-xiaomi-20220406165602534.chn', 'content': 'Từ ngày 6/4 - 10/4, FPT Shop giảm đến 45% cho sản phẩm thuộc hệ sinh thái Xiaomi Theo đó, từ nay đến hết ngày 10/04/202, FPT Shop mạnh tay giảm đến 45% cho nhiều sản phẩm thông minh IoT thuộc hệ sinh thái Xiaomi. Cụ thể, FPT Shop giảm đến 3.000.000 đồng cho máy lọc không khí, robot hút bụi Xiaomi với giá còn từ 2.490.000 đồng. Bên cạnh đó, các sản phẩm vòng đeo tay Mi Smart Band 5, loa Bluetooth Xiaomi, chuột không dây, Mi Smart Band, camera an ninh, nồi chiên không dầu, đồng hồ thông minh Xiaomi... giảm đến 45%, giá nhẹ nhàng chỉ từ 240.000 đồng. Hấp dẫn hơn, khách hàng chọn mua tai nghe Xiaomi Buds 3, Buds 3T Pro tại FPT Shop còn được tặng thêm cân Xiaomi và loa trị giá đến 1.180.000 đồng. Không kém thiết thực, khách hàng đặt trước đồng hồ Xiaomi Watch S1 Active còn nhận được bộ quà trị giá 2.180.000 đồn

In [5]:
client = Elasticsearch("http://localhost:9200")

client.info()


/home/hplong/.local/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'HoangPhiLong-20182651',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'jF2zfdDdQqueQik2u6h9Ug',
 'version': {'number': '7.17.2',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': 'de7261de50d90919ae53b0eff9413fd7e5307301',
  'build_date': '2022-03-28T15:12:21.446567561Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [6]:
def gendata():
    for key,value in kenhContent.items():
         for item in value:
            temp_item = {k: item[k] for k in set(list(item.keys())) - set(['id'])}
            temp_item['category'] = key
            client.index(index="journal", id=item['id'], document=temp_item)

gendata()